In [102]:
import cv2
import numpy as np

def remove_lines_inpaint(img, lines):
    mask = cv2.dilate(lines, np.ones((3,3), np.uint8), iterations=1)
    return cv2.inpaint(img, mask, 3, cv2.INPAINT_TELEA)

def preserve_text(original, processed):
    diff = cv2.absdiff(original, processed)
    _, mask = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)
    return cv2.bitwise_and(original, original, mask=mask) + processed


def remove_lines(img, vertical_scale=80, horizontal_scale=90, dilation_iter=1):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, bw = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)

    
    # Define structure for vertical line detection
    rows = bw.shape[0]
    vertical_size = int(rows / vertical_scale)
    verticalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (1, vertical_size))
    
    # Detect vertical lines
    vertical_lines = cv2.erode(bw, verticalStructure)
    vertical_lines = cv2.dilate(vertical_lines, verticalStructure, iterations=dilation_iter)
    
    # Define structure for horizontal line detection
    cols = bw.shape[1]
    horizontal_size = int(cols / horizontal_scale)
    horizontalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (horizontal_size, 1))
    
    # Detect horizontal lines
    horizontal_lines = cv2.erode(bw, horizontalStructure)
    horizontal_lines = cv2.dilate(horizontal_lines, horizontalStructure, iterations=dilation_iter)
    
    # Combine vertical and horizontal lines
    lines = cv2.add(vertical_lines, horizontal_lines)
    
    # Subtract lines from the original binary image to remove them
    # filtered_image = cv2.subtract(bw, lines)
    filtered_image = remove_lines_inpaint(cv2.bitwise_not(bw), lines)
    # kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    # filtered_image = cv2.filter2D(filtered_image, -1, kernel)
    
    # Optionally, apply a bitwise not to invert the colors back
    # filtered_image = cv2.bitwise_not(filtered_image)
    # result = preserve_text(cv2.bitwise_not(bw), filtered_image)
    
    return filtered_image

# Example usage:
filename = r'C:\Users\arajaram\OneDrive - Maryland Department of Transportation(MDOT)\Desktop\imgs\Image0000003-0000.tif'
img = cv2.imread(filename)
filtered_img = remove_lines(img, vertical_scale=30, horizontal_scale=30, dilation_iter=1)
cv2.imwrite('filtered_image.png', filtered_img)


True